<table width="100%" style="border: none; border-collapse: collapse;">
  <tr>
    <td style="vertical-align: middle; padding: 10px;">
      <h1 style="margin: 0; font-size: 1.5em;">Guía Semana No. 2</h1>
    </td>
    <td style="text-align: right; vertical-align: middle; padding: 10px;">
      <img src="resources_5082025/University_of_Los_Andes_logo.png" width="120" alt="Logo Universidad de los Andes" />
    </td>
  </tr>
</table>
<hr>

Esta guía forma parte del trabajo de la segunda semana del curso y tiene como objetivo introducir herramientas simbólicas y numéricas para la modelación y análisis de sistemas mecánicos.

El contenido está organizado para abordar:

1. **Marcos de referencia simbólicos con SymPy Mechanics**: cómo definir sistemas de coordenadas móviles e inerciales y cómo describir posiciones y orientaciones de forma simbólica.
2. **Cinemática directa e inversa de robots planos 2R**: formulación matemática y obtención de soluciones analíticas y numéricas.

A lo largo de los ejercicios, se utilizarán librerías de Python para trabajar con álgebra simbólica, animaciones y visualización de resultados.

## Importación de librerías necesarias

Se importan las librerías que se utilizarán en la guía:

- **NumPy**: operaciones matemáticas y matriciales numéricas.
- **Matplotlib**: para realizar gráficas y animaciones.
- **SymPy**: para cálculos simbólicos y álgebra matemática.
- **SymPy Physics Mechanics**: para trabajar con conceptos de mecánica como puntos, vectores y marcos de referencia.

También se habilita la impresión de expresiones matemáticas usando LaTeX para mejorar la legibilidad.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sympy import symbols, Matrix, pi, cos, sin, simplify, eye, solve, latex, atan2, pprint, init_printing, Derivative, sqrt
from sympy.physics.mechanics import ReferenceFrame, dynamicsymbols, init_vprinting, Point

# Configuración de impresión en formato LaTeX (MathJax)
init_vprinting(use_latex='mathjax')

## Definición de variables simbólicas y marcos de referencia

En esta sección se definen:

- Variables escalares constantes y dependientes del tiempo.
- Marcos de referencia:
  - `N`: marco inercial (fijo en el espacio).
  - `A`: marco rotado respecto a `N` alrededor del eje Z.

Esto servirá como base para describir movimientos y orientaciones de cuerpos rígidos.

In [6]:
# Variables constantes y dependientes del tiempo
t, l_0 = symbols('t l_0')
l1, l2 = dynamicsymbols('l1 l2')
theta1, theta2 = dynamicsymbols('theta1 theta2') #q1 y q2, angulos

# Definición de marcos de referencia
N = ReferenceFrame("N") #sistema de referencial global, lo ponemos en el origen del primer joint
A = N.orientnew("A", "Axis", [theta1, N.z])#Definimos nuestro sistema A, que se encuentra en el segundo joint, este esta rotando con respecto a N
B = A.orientnew("B", "Axis", [theta2, A.z]) #Definimos nuestro sistema B, que se encuentra en el tercer joint, este esta rotando con respecto a A

#Si se multiplica la matriz de A con respecto a N, y otra matriz de B con respecto a A,
#puedo hallar la matriz de B con respecto a N

#Queremos saber la posición de un punto
O = Point('O') #origen del sistema de referencia N
P1 = O.locatenew('P1', l1 * A.x) #Punto P1 en el sistema de referencia N , AX vector unitario de A, en este punto es donde se encuentra el segundo joint
P2 = P1.locatenew('P2', l2 * B.x) #Punto P2 en el sistema de referencia N, este es el punto donde se encuentra el tercer joint

efector_final_pos = P2.pos_from(O).express(N).simplify() #toma a P2, lo toma desde el origen O, lo expresa con respecto a N y lo simplifica
print("\n Posicion del efector final:")
pprint(efector_final_pos) #esto es la posicion final, expresada con respecto a N,

def cinematica_directa(theta1_val, theta2_val, l1_val, l2_val): #poner los numeros que vamos a utilizar, aqui ya deja de ser simbolico
    subs_dict = {
        theta1: np.deg2rad(theta1_val),
        theta2:  np.deg2rad(theta2_val),
        l1: l1_val,
        l2: l2_val
    }
    pos = efector_final_pos.subs(subs_dict) #utilizamos la funcion subtituir y se lo mandamos al efector, lista con numeros, [x,y]
    #lo que estamos haciendo aqui es dado los angulos (el segundo relativo al primero) y los valores de las longitudes, hallamos la posicion final en la que queda el brazo
    return (pos.dot(N.x), float(pos.dot(N.y))) #lo pone en terminos del sistema coordenado, para expresarlo vectorialmente
print(cinematica_directa(45, 0, 1, 1))


 Posicion del efector final:
(l₁(t)⋅cos(θ₁(t)) + l₂(t)⋅cos(θ₁(t) + θ₂(t))) n_x + (l₁(t)⋅sin(θ₁(t)) + l₂(t)⋅ ↪

↪ sin(θ₁(t) + θ₂(t))) n_y
(1.41421356237310, 1.414213562373095)


## Ejercicio No. 1: Independencia lineal y análisis en el tiempo ejercicio No. 4

Objetivo:

1. Crear una función que reciba el tiempo y calcule las aceleraciónes y velocidades para poder obtener la cinemática.

## Ejercicio No. 2: Cinemática directa de un robot planar 2R

En este ejercicio se modela un robot plano con dos articulaciones rotacionales (2R) usando SymPy Mechanics.

Pasos:
1. Definir marcos de referencia para cada articulación.
2. Establecer la posición de los puntos relevantes (origen, articulaciones y efector final).
3. Calcular la posición del efector respecto al marco inercial `N`.
4. Evaluar la posición para un conjunto de valores numéricos.

Este resultado servirá de base para el cálculo de la cinemática inversa.

In [ ]:
import sympy as sp
from sympy.physics.mechanics import ReferenceFrame, Point, dynamicsymbols, mechanics_printing

# Habilitar impresión simbólica sin formato bonito (para depuración)
mechanics_printing(pretty_print=False)

# Variables simbólicas
theta1, theta2 = dynamicsymbols('theta1 theta2')
L1, L2 = sp.symbols('L1 L2')

# Marcos de referencia
N = ReferenceFrame('N')
A = N.orientnew('A', 'Axis', [theta1, N.z])
B = A.orientnew('B', 'Axis', [theta2, A.z])

# Puntos relevantes
O = Point('O')
P1 = O.locatenew('P1', L1 * A.x)
P2 = P1.locatenew('P2', L2 * B.x)

# Vector de posición del efector
vector_pos_efector = P2.pos_from(O).express(N).simplify()
print("\nVector de posición del efector (en marco N):")
sp.pprint(vector_pos_efector)

# Evaluación numérica de ejemplo
valores = {
    theta1: sp.rad(45),
    theta2: sp.rad(30),
    L1: 5,
    L2: 3
}

vector_numerico = vector_pos_efector.subs(valores).evalf()
x = vector_numerico.dot(N.x)
y = vector_numerico.dot(N.y)
print(f"\nPosición numérica del efector:\nx = {x:.4f}\ny = {y:.4f}")

## Ejercicio No. 3: Mecanismo Slider-Crank

Objetivos:
- Graficar la posición, velocidad y aceleración del mecanismo tipo biela-manivela (slider-crank) a lo largo del tiempo.
- Adicionalmente, graficar la fuerza generada en función de la geometría del mecanismo.

Este ejercicio combina análisis cinemático con análisis dinámico.

## Ejercicio No. 4: Análisis cinemático avanzado

Trabajando a partir del Ejercicio 2, se pide:
- Graficar la posición y velocidad del "pin" en el marco B.
- Graficar las velocidades y aceleraciones del "pin" en el marco A.

Este ejercicio permite profundizar en el cálculo de derivadas vectoriales y en la interpretación de magnitudes en distintos marcos de referencia.